In [2]:
import pandas as pd
import os
import numpy as np
data_folder_path = 'data'

In [4]:

def combine_dfs(df1, df2):
  matching_cols = set(df1.columns) & set(df2.columns)
  if not matching_cols:
    combined_df = pd.concat([df1, df2], axis=1, join='outer')
  else:
    combined_df = pd.concat([df1, df2], axis=0)
  combined_df = combined_df.loc[~combined_df.index.duplicated(keep='first')]
  return combined_df
   

stock3 = ['BAC', 'TSLA', 'AAPL']
combined_df = pd.DataFrame()
for stock in stock3:
    stock_df = pd.DataFrame() 
    for filename in os.listdir(data_folder_path):
        if filename.endswith(".csv") and stock in filename and 'combined' not in filename:
            file_path = os.path.join(data_folder_path, filename)
            df = pd.read_csv(file_path)
            df.set_index('Time', inplace=True)
            stock_df = combine_dfs(stock_df, df)
        else:
            continue
    combined_df = combine_dfs(combined_df, stock_df)
        
    
combined_df.fillna(0, inplace=True)
combined_df.sort_index(inplace=True)
combined_df.to_csv(os.path.join(data_folder_path, 'combined', 'combined.csv'), index=True)

#for i in range(len(dfs)):
##    if i == 0:
#        continue
#    else:
#        dfs[0] = pd.merge(dfs[0], dfs[i], how='outer')
#combined_df = pd.merge(dfs, how='left')


In [5]:
#read combined csv
#convert everything into a percent change from the previous line
#clip to -1, 1
#remove the first line
#save as npy file
combined_csv_path = os.path.join(data_folder_path, 'combined/combined.csv')

df = pd.read_csv(combined_csv_path)

percent_change_df = df.iloc[:, 1:].pct_change()

percent_change_df.clip(-1, 1, inplace=True)

result_df = pd.concat([df.iloc[:, :1], percent_change_df], axis=1)

percent_change_df = result_df.iloc[1:]  

final_df = percent_change_df.fillna(0)

final_df.to_csv('full_data.csv', index=False)